In [1]:
  
# SKT Brain의 KoBERT, 학습 및 테스트 데이터셋만 따로 준비 = 카카오 브런치[text, label]
# SKT Brain github 주소는 다음과 같습니다. https://github.com/SKTBrain/KoBERT

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup


     |████████████████████████████████| 46.9 MB 1.3 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595722 sha256=c72ff596141fc622e2dc30ef5683e07e6a7edf62db513dad66bf22595b378c72
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.3 MB/s 
     |████████████████████████████████| 769 kB 4.4 MB/s 
     |████████████████████████████████| 3.0 MB 38.8 MB/s 
     |████████████████████████████████| 895 kB 54.8 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-g3mphjto
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pi

In [2]:
# GPU 사용 시
#device = torch.device("cuda:0")
device = torch.device("cpu")

# BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [3]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# 데이터셋 불러오기
import pandas as pd
data = pd.read_excel('/content/drive/MyDrive/데캡디/감성대화/Training/train.xlsx')

In [10]:
# data 확인
data.head()

,번호,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,44164,청년,남성,"연애, 결혼, 출산",해당없음,기쁨,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,아 지금 정말 신이 나.,잘 된 일이네요.,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.,NaN,NaN
1,3926,노년,남성,"건강, 죽음",만성질환 유,불안,스트레스 받는,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.,현재 상황에서 변화를 주기 위해 어떻게 하면 될까요?,약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.,약을 잘 챙겨 먹어 건강해지시길 바랄게요.,NaN,NaN
2,50882,청소년,여성,학업 및 진로,해당없음,당황,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.,아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼.,이 상황에서 할 수 있는 게 어떤 것이 있을까요?,중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야.,수업을 어떤 방식으로 열심히 듣는게 좋을까요?,선생님이 강조하신 부분을 필기하고 집에서 매일 수업 내용을 복습하려고 해.,열심히 해서 좋은 결과가 있었으면 좋겠어요.
3,31303,노년,남성,재정,만성질환 무,기쁨,신이 난,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 후 첫 월급이라 정말 기쁘시겠어요.,퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 ...,이 기분을 유지하려면 어떻게 해야 할까요?,회사생활을 열심히 해서 계속 월급을 받을거야!,계속 원하시는 대로 회사생활이 이어지길 바랄게요.,NaN,NaN
4,47200,노년,여성,재정,만성질환 유,기쁨,안도,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶으시군요.,NaN,NaN,NaN,NaN


In [11]:
# data column 확인
data.columns

Index(['번호', '연령', '성별', '상황키워드', '신체질환', '감정_대분류', '감정_소분류', '사람문장1',
       '시스템응답1', '사람문장2', '시스템응답2', '사람문장3', '시스템응답3', '사람문장4', '시스템응답4'],
      dtype='object')

In [13]:
data['감정_대분류'].unique()

array(['기쁨', '불안', '당황', '슬픔', '분노', '상처', '불안 ', '기쁨 '], dtype=object)

In [14]:
data['감정_소분류'].unique()

array(['신이 난', '스트레스 받는', '당황', '안도', '취약한', '우울한', '구역질 나는', '좌절한',
       '눈물이 나는', '배신당한', '남의 시선을 의식하는', '죄책감의', '당혹스러운', '분노', '두려운',
       '낙담한', '회의적인', '마비된', '혼란스러운', '흥분', '한심한', '염세적인', '안달하는',
       '걱정스러운', '조심스러운', '충격 받은', '상처', '불안', '노여워하는', '기쁨', '짜증내는',
       '환멸을 느끼는', '비통한', '혐오스러운', '억울한', '만족스러운', '부끄러운', '버려진', '고립된',
       '괴로워하는', '자신하는', '외로운', '방어적인', '편안한', '질투하는', '감사하는', '초조한',
       '실망한', '슬픔', '신뢰하는', '가난한, 불우한', '후회되는', '툴툴대는', '성가신', '열등감',
       '희생된', '느긋', '악의적인'], dtype=object)

In [16]:
df = data[ ['감정_대분류', '사람문장1'] ]

In [17]:
df.head()

,감정_대분류,사람문장1
0,기쁨,아내가 드디어 출산하게 되어서 정말 신이 나.
1,불안,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.
2,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.
3,기쁨,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.
4,기쁨,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.


In [22]:
# 데이터 합치기
new_data = df[['사람문장1', '감정_대분류']]
new_data = new_data.rename(columns={'사람문장1':'content', '감정_대분류':'label'})
new_data.head()

,content,label
0,아내가 드디어 출산하게 되어서 정말 신이 나.,기쁨
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,불안
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,당황
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,기쁨
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기쁨


In [23]:
# label encoding
new_data.loc[(new_data['label'] == "기쁨"), 'label'] = 0  #발라드 => 0
new_data.loc[(new_data['label'] == "불안"), 'label'] = 1  #밤 => 0
new_data.loc[(new_data['label'] == "당황"), 'label'] = 2  #슬픈 => 0
new_data.loc[(new_data['label'] == "슬픔"), 'label'] = 3  #발라드 => 0
new_data.loc[(new_data['label'] == "분노"), 'label'] = 4  #밤 => 0
new_data.loc[(new_data['label'] == "상처"), 'label'] = 5  #슬픈 => 0
new_data.loc[(new_data['label'] == "불안 "), 'label'] = 1  #슬픈 => 0
new_data.loc[(new_data['label'] == "기쁨 "), 'label'] = 0  #슬픈 => 0

# list형태로 데이터 저장 -> [content, label] 형태
data_list = []
for q, label in zip(new_data['content'], new_data['label'])  :
    data = []
    data.append(str(q)) # str타입으로 변환, 안 해주면 나중에 타입변환할 때 오류남
    data.append(str(label))

    data_list.append(data)

In [25]:
# 데이터 확인
print(data_list[0])
print(data_list[-1])

['아내가 드디어 출산하게 되어서 정말 신이 나.', '0']
['친구 관계가 너무 힘들어. 베푸는 만큼 돌아오지 않는 것 같아.', '1']


In [26]:
# train, test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [27]:
# 데이터 개수 확인
print(len(dataset_train))
print(len(dataset_test))

30659
10220


In [28]:
# KoBERT모델의 입력으로 들어갈 수 있는 형태가 되도록 변환해주는 class
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        # BERTSentenceTransform이라는 모듈 사용
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        # 타입 변환
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    # item 가져오기
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    # 길이 출력
    def __len__(self):
        return (len(self.labels))

In [29]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [30]:
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# 패딩
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model


In [31]:
# 데이터 확인 -> array1은 시퀀스, array2는 길이와 타입, array3은 attention mask 시퀀스(1로 패딩된 값들은 연산할 필요가 없기 때문에, 연산을 하지 않아도 됨 -> 이걸 알려주는 데이터가 바로 얘)
data_train[0]

(array([   2, 4688, 6797, 5940,  950, 7795, 4384, 7088, 2573, 5794, 5439,
        5007, 5112, 5330, 1370,  517,   54,    3,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(18, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 4)

In [32]:
# torch 형식의 dataset 생성
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [33]:
# KoBERT 학습모델
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,   # 클래스 수 조정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [34]:
# BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio) # 초기에 성능을 끌어올리는 것 = 운동할 때 워밍업 느낌

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0] # 맞은 비율
    return train_acc
    
train_dataloader

In [ ]:
# 모델 학습시키기
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/480 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.855034351348877 train acc 0.125
epoch 1 batch id 201 loss 1.2668131589889526 train acc 0.34172885572139305
epoch 1 batch id 401 loss 1.0473443269729614 train acc 0.46758104738154616
epoch 1 train acc 0.4892144097222222


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 batch id 1 loss 0.8429675102233887 train acc 0.703125
epoch 4 batch id 201 loss 0.9809877276420593 train acc 0.7308768656716418
epoch 4 batch id 401 loss 0.7241630554199219 train acc 0.7537406483790524
epoch 4 train acc 0.7562174479166667


  0%|          | 0/160 [00:00<?, ?it/s]

epoch 4 test acc 0.658016690340909


  0%|          | 0/480 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.754875659942627 train acc 0.734375


In [ ]:
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# 새로운 문장 테스트
def predict(predict_sentence):
  data = [predict_sentence, '0']
  dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
      token_ids = token_ids.long().to(device)
      print("token_ids : ", token_ids)
      segment_ids = segment_ids.long().to(device)
      print("segment_ids : ", segment_ids)

      valid_length= valid_length
      print("valid_length : ", valid_length)
      label = label.long().to(device)
      print("label : ", label)
      out = model(token_ids, valid_length, segment_ids)

      print("out : ", out)

      test_eval=[]
      for i in out:
          logits=i
          logits = logits.detach().cpu().numpy()
          print("logits : ", logits)

          if np.argmax(logits) == 0:
                test_eval.append("발라드가")
          elif np.argmax(logits) == 1:
                test_eval.append("밤이")
          elif np.argmax(logits) == 2:
                test_eval.append("슬픔이")

      print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

using cached model


In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 슬퍼요


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


token_ids :  tensor([[   2, 2948, 7706, 6999,    3,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1]], device='cuda:0')
segment_ids :  tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0')
valid_length :  tensor([5], dtype=torch.int32)
label :  tensor([0], device='cuda:0')
out :  tensor([[-0.4484,  0.4338, -0.2465]], device='cuda:0', grad_fn=<AddmmBackward>)
logits :  [-0.44841397  0.43383607 -0.2465297 ]
>> 입력하신 내용에서 밤이 느껴집니다.




KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), '/content/drive/My Drive/nlp/model_state_dict.pt')

In [ ]:
torch.save(model, '/content/drive/My Drive/nlp/model.pt')